In [78]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict, OrderedDict

In [152]:
class Eval:
    
    def __init__(self, relevant_docs_file):
        self.relevant_docs = self.read_relevant_docs(relevant_docs_file)
        self.current_system_results = None
    
    @staticmethod
    def parse_relevant_doc(line):
        """parse each line of qrels.txt into usable information"""
        query_num = re.findall(r'([0-9]+?):', line)
        relevant_docs_n_scores = re.findall(r'\(([0-9]+?),([0-9]+?)\)', line)
        return query_num[0], relevant_docs_n_scores
    
    @classmethod
    def read_relevant_docs(cls, relevant_docs_file):
        """read relevant documents and the associated relevance scores for all queries"""
        with open(relevant_docs_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        results = dict()
        for line in lines:
            query_num, relevant_docs_n_scores = cls.parse_relevant_doc(line)
            #print(relevant_docs_n_scores)
            results[int(query_num)] = [(int(item[0]), int(item[1])) for item in relevant_docs_n_scores]
        return results
    
    @staticmethod
    def parse_system_result(line):
        """parse each line of system results into usable information"""
        pa = r'([0-9]+?) 0 ([0-9]+?) ([0-9]+?) (.+?) 0'
        sys_rlt = re.findall(pa, line)
        query_num, doc_num, rank, score = sys_rlt[0]
        return query_num, doc_num, rank, score
    
    @classmethod
    def read_system_results(cls, sys_rlts_file):
        """read and parse system results file"""
        with open(sys_rlts_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        results = defaultdict(list)
        for line in lines:
            query_num, doc_num, rank, score = cls.parse_system_result(line)
            results[int(query_num)] += [(int(rank), int(doc_num), float(score))]
        print('Loading system results: {}...'.format(sys_rlts_file))
        return results
    
    def p_at_N(self, query_num, N=10):
        """calculate precision @ N for a query"""
        N_docs = self.current_system_results[query_num][:N]  # the sys results have been sorted
        relevant_doc_nums = [item[0] for item in self.relevant_docs[query_num]]
        count = 0
        for _, doc_num, _ in N_docs:
            if doc_num in relevant_doc_nums:
                count += 1
        return count / N
    
    def r_at_N(self, query_num, N=50):
        """calculate recall @ N for a query"""
        N_docs = self.current_system_results[query_num][:N]  # the sys results have been sorted
        relevant_doc_nums = [item[0] for item in self.relevant_docs[query_num]]
        count = 0
        for _, doc_num, _ in N_docs:
            if doc_num in relevant_doc_nums:
                count += 1
        return count / len(relevant_doc_nums)
    
    def r_precision(self, query_num):
        """calculate precision  for a query"""
        relevant_doc_nums = [item[0] for item in self.relevant_docs[query_num]]
        rank = len(relevant_doc_nums)
        R_docs = self.current_system_results[query_num][:rank]
        count = 0
        for _, doc_num, _ in R_docs:
            if doc_num in relevant_doc_nums:
                count += 1
        return count / rank

In [153]:
e = Eval(relevant_docs_file='systems/qrels.txt')

In [154]:
e.current_system_results = e.read_system_results('systems/S1.results')

Loading system results: systems/S1.results...


In [155]:
for i in range(10):
    print(e.p_at_N(i+1))

0.4
0.3
0.0
0.6
0.2
0.7
0.2
0.6
0.9
0.0


In [156]:
for i in range(10):
    print(e.r_at_N(i+1))

0.6666666666666666
1.0
1.0
0.875
0.42857142857142855
1.0
0.6666666666666666
1.0
0.9
0.8


In [157]:
for i in range(10):
    print(e.r_precision(i+1))

0.16666666666666666
0.25
0.0
0.7
0.2857142857142857
0.75
0.3333333333333333
0.625
0.9
0.0
